### Check Dependencies

In [23]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai langchain_mistralai neo4j tiktoken

# !pip3 show langchain neo4j langchain_mistralai

### Load environment variables

In [24]:
from dotenv import load_dotenv
import os
load_dotenv()

print("NEO4J_URI = " + os.getenv("NEO4J_URI"))
print("NEO4J_USERNAME = " + os.getenv("NEO4J_USERNAME"))
print("NEO4J_PASSWORD = " + os.getenv("NEO4J_PASSWORD"))

NEO4J_URI = bolt://localhost:7687
NEO4J_USERNAME = neo4j
NEO4J_PASSWORD = neo4j123


### Create Neo4jGraph

Make sure the Neo4j instance is running and the environment variables are set correctly.

In [25]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()
# graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="neo4j123")


### Check graph schema

In [26]:
graph.schema

'Node properties are the following:\nPerson {name: STRING, tags: STRING, description: STRING},SoftwareSystem {description: STRING, tags: STRING, name: STRING},Container {name: STRING, tags: STRING, description: STRING},Component {name: STRING, tags: STRING, description: STRING}\nRelationship properties are the following:\nUses {description: STRING}\nThe relationships are the following:\n(:Person)-[:Uses]->(:Container),(:Person)-[:Uses]->(:SoftwareSystem),(:Person)-[:Uses]->(:Person),(:SoftwareSystem)-[:Uses]->(:Person),(:SoftwareSystem)-[:Uses]->(:SoftwareSystem),(:SoftwareSystem)-[:Contains]->(:Container),(:Container)-[:Uses]->(:Component),(:Container)-[:Uses]->(:Container),(:Container)-[:Uses]->(:SoftwareSystem),(:Container)-[:Contains]->(:Component),(:Component)-[:Uses]->(:Component),(:Component)-[:Uses]->(:Container),(:Component)-[:Uses]->(:SoftwareSystem)'

In [27]:
graph.structured_schema

{'node_props': {'Person': [{'property': 'name', 'type': 'STRING'},
   {'property': 'tags', 'type': 'STRING'},
   {'property': 'description', 'type': 'STRING'}],
  'SoftwareSystem': [{'property': 'description', 'type': 'STRING'},
   {'property': 'tags', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'}],
  'Container': [{'property': 'name', 'type': 'STRING'},
   {'property': 'tags', 'type': 'STRING'},
   {'property': 'description', 'type': 'STRING'}],
  'Component': [{'property': 'name', 'type': 'STRING'},
   {'property': 'tags', 'type': 'STRING'},
   {'property': 'description', 'type': 'STRING'}]},
 'rel_props': {'Uses': [{'property': 'description', 'type': 'STRING'}]},
 'relationships': [{'start': 'Person', 'type': 'Uses', 'end': 'Container'},
  {'start': 'Person', 'type': 'Uses', 'end': 'SoftwareSystem'},
  {'start': 'Person', 'type': 'Uses', 'end': 'Person'},
  {'start': 'SoftwareSystem', 'type': 'Uses', 'end': 'Person'},
  {'start': 'SoftwareSystem', 'type': 'Uses', 'end

### Create Cypher Query

In [28]:
## What are the persons using systems?
graph.query("MATCH (n:Person) RETURN n")

[{'n': {'name': 'Personal Banking Customer',
   'description': 'A customer of the bank, with personal bank accounts.',
   'tags': 'Element,Person,Customer'}},
 {'n': {'name': 'Customer Service Staff',
   'description': 'Customer service staff within the bank.',
   'tags': 'Element,Person,Bank Staff'}},
 {'n': {'name': 'Back Office Staff',
   'description': 'Administration and support staff within the bank.',
   'tags': 'Element,Person,Bank Staff'}}]

In [29]:
## What software systems are used by customers?
graph.query("""
    MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
    WHERE p.tags CONTAINS "Customer"
    RETURN DISTINCT ss.name
    """)

[{'ss.name': 'ATM'}, {'ss.name': 'Internet Banking System'}]

In [8]:
## What software systems are used by customers to withdraw cash?
graph.query("""
    MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
    WHERE p.tags CONTAINS "Customer" and ss.description CONTAINS "withdraw cash"
    RETURN DISTINCT ss.name
    """)

[{'ss.name': 'ATM'}]

### Create LLM

In [9]:
# from langchain_openai import ChatOpenAI
# os.environ["OPENAI_API_KEY"] = .......
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

from langchain_mistralai.chat_models import ChatMistralAI
llm = ChatMistralAI(mistral_api_key=os.getenv("MISTRAL_API_KEY"), model=os.getenv("MISTRAL_MODEL"), temperature=0)

print("MISTRAL_MODEL = " + os.getenv("MISTRAL_MODEL"))

MISTRAL_MODEL = open-mistral-7b


### Create GraphCypherQAChain

Please refer to API documentation for more details.

https://api.python.langchain.com/en/latest/chains/langchain.chains.graph_qa.cypher.GraphCypherQAChain.html

In [10]:
from langchain.chains import GraphCypherQAChain
#  input_key: str = "query"  
#  output_key: str = "result" 

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
# top_k=2
# return_intermediate_steps=True
# return_direct=True


# class GraphCypherQAChain(Chain):
#     graph: GraphStore = Field(exclude=True)
#     cypher_generation_chain: LLMChain
#     qa_chain: LLMChain
#     graph_schema: str
#     input_key: str = "query"  #: :meta private:
#     output_key: str = "result"  #: :meta private:
#     top_k: int = 10
#     """Number of results to return from the query"""
#     return_intermediate_steps: bool = False
#     """Whether or not to return the intermediate steps along with the final answer."""
#     return_direct: bool = False
#     """Whether or not to return the result of querying the graph directly."""
#     cypher_query_corrector: Optional[CypherQueryCorrector] = None
#     """Optional cypher validation tool"""

# from_llm(
#         cls,
#         llm: Optional[BaseLanguageModel] = None,
#         *,
#         qa_prompt: Optional[BasePromptTemplate] = None,
#         cypher_prompt: Optional[BasePromptTemplate] = None,
#         cypher_llm: Optional[BaseLanguageModel] = None,
#         qa_llm: Optional[BaseLanguageModel] = None,
#         exclude_types: List[str] = [],
#         include_types: List[str] = [],
#         validate_cypher: bool = False,
#         qa_llm_kwargs: Optional[Dict[str, Any]] = None,
#         cypher_llm_kwargs: Optional[Dict[str, Any]] = None,
#         **kwargs: Any,
#     ) -> GraphCypherQAChain:

### Check the default prompts


In [11]:
from langchain.chains.graph_qa.prompts import CYPHER_GENERATION_PROMPT, CYPHER_QA_PROMPT

print("CYPHER_GENERATION_PROMPT = ", flush=True)
print(CYPHER_GENERATION_PROMPT)

print("CYPHER_QA_PROMPT = ", flush=True)
print(CYPHER_QA_PROMPT)

CYPHER_GENERATION_PROMPT = 
input_variables=['question', 'schema'] template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'
CYPHER_QA_PROMPT = 
input_variables=['context', 'question'] template="You are an assistant that helps to form nice and human understandable answers.\nThe information part contains the provided information that you must use to construct an answer.\nThe provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.\nMake the answer sound as a response 

### Ask question to QA chain

The **question** must be passed as a dictionary with a key "query" and the value is the question.

The QA chain will generate a cypher query statement using `CYPHER_GENERATION_PROMPT` and return the answer to the question.


In [12]:
response = chain.invoke({"query": "What software systems are used by Back Office Staff?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Back Office Staff"})-[:Uses]->(ss:SoftwareSystem)
RETURN ss.name

Full Context:
[{'ss.name': 'Mainframe Banking System'}]

> Finished chain.


{'query': 'What software systems are used by Back Office Staff?',
 'result': 'The Back Office Staff uses the Mainframe Banking System.'}

In [13]:
response = chain.invoke({"query": "What software systems are used by Personal Banking Customer?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Personal Banking Customer"})-[:Uses]->(ss:SoftwareSystem)
RETURN ss.name

Full Context:
[{'ss.name': 'ATM'}, {'ss.name': 'Internet Banking System'}]

> Finished chain.


{'query': 'What software systems are used by Personal Banking Customer?',
 'result': 'Personal Banking Customers use ATM and Internet Banking System software systems.'}

In [14]:
response = chain.invoke({"query": "What software systems are used by customers?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
RETURN ss.name

Full Context:
[{'ss.name': 'ATM'}, {'ss.name': 'Internet Banking System'}, {'ss.name': 'Mainframe Banking System'}, {'ss.name': 'Mainframe Banking System'}]

> Finished chain.


{'query': 'What software systems are used by customers?',
 'result': 'Customers use ATM, Internet Banking System, and Mainframe Banking System software systems.'}

### Promt Refinement

https://python.langchain.com/docs/integrations/graphs/memgraph/#prompt-refinement


In [15]:
from langchain_core.prompts import PromptTemplate

# ## Refine the CYPHER_GENERATION_TEMPLATE by adding examples

MY_CYPHER_GENERATION_TEMPLATE = """
Task: Generate a syntactically correct cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Here are a few examples of generated Cypher statements for particular questions:
# What software systems are used by Bank Staff?
MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
WHERE p.tags CONTAINS "Bank Staff"
RETURN DISTINCT ss.name
# What software systems are used by customers to withdraw cash?
MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
WHERE p.tags CONTAINS "Customer" and ss.description CONTAINS "withdraw cash"
RETURN DISTINCT ss.name
    
The question is:
{question}
"""

MY_CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=MY_CYPHER_GENERATION_TEMPLATE
)

## Use the default CYPHER_GENERATION_PROMPT
# MY_CYPHER_GENERATION_PROMPT = CYPHER_GENERATION_PROMPT

## Use the default CYPHER_QA_PROMPT
# MY_CYPHER_QA_PROMPT = CYPHER_QA_PROMPT
MY_CYPHER_QA_TEMPLATE = """
You are an assistant that helps to form nice and human understandable answers.
The context part contains the provided information that you must use to construct an answer.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. 
Do not mention that you based the result on the given context.
Do not make up anything which does not exist in the provided context.

Here is an example:
Question: What software systems are used by customers?
Context: ATM, Internet Banking System
Helpful Answer: ATM and Internet Banking System.

Follow this example when generating answers. If the provided context is empty, say that you don't know the answer.

Context: {context}

Question: {question}
Helpful Answer:
"""

MY_CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=['context', 'question'], template=MY_CYPHER_QA_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, qa_prompt=MY_CYPHER_QA_PROMPT,cypher_prompt=MY_CYPHER_GENERATION_PROMPT,verbose=True)





> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
WHERE p.tags CONTAINS "Customer"
RETURN DISTINCT ss.name
Full Context:
[{'ss.name': 'ATM'}, {'ss.name': 'Internet Banking System'}]

> Finished chain.


{'query': 'What software systems are used by customers?',
 'result': 'Customers use ATM, Internet Banking System, and Mainframe Banking System software systems.'}

In [ ]:
responseresponse = chain.invoke({"query": "What software systems are used by customers?"})
response

In [17]:
responseresponse = chain.invoke({"query": "What systems can be used by customers to withdraw cash?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:Uses]->(ss:SoftwareSystem)
WHERE p.tags CONTAINS "Customer" and ss.description CONTAINS "withdraw cash"
RETURN DISTINCT ss.name
Full Context:
[{'ss.name': 'ATM'}]

> Finished chain.


{'query': 'What software systems are used by customers?',
 'result': 'Customers use ATM, Internet Banking System, and Mainframe Banking System software systems.'}

### Few-shot examples
    

In [51]:
examples = [
    {
        "question": "who are the users",
        "query": "MATCH (p:Person) RETURN distinct p",
    },
    {
        "question": "what software systems are used by customers",
        "query": "MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Customer' RETURN DISTINCT s.name,s.description",
    },
    {
        "question": "what software systems are used by customers to withdraw cash",
        "query": "MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Customer' AND s.description CONTAINS 'withdraw cash' RETURN DISTINCT s.name,s.description",
    },
    {
        "question": "what software systems are available",
        "query": "MATCH (s:SoftwareSystem) RETURN DISTINCT s.name,s.description",
    }
]

In [52]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)

promt_prefix = """
Task: Generate a syntactically correct cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Here are a few examples of generated Cypher statements for particular questions:
"""

promt_suffix="""
User input: {question}
Cypher query:
"""

fewShotPrompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=promt_prefix,
    suffix=promt_suffix,
    input_variables=["question", "schema"],
)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, cypher_prompt=fewShotPrompt, verbose=True)


In [53]:
print(fewShotPrompt.format(question="What software systems are used by customers to withdraw cash?", schema=graph.structured_schema))


Task: Generate a syntactically correct cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{'node_props': {'Person': [{'property': 'name', 'type': 'STRING'}, {'property': 'tags', 'type': 'STRING'}, {'property': 'description', 'type': 'STRING'}], 'SoftwareSystem': [{'property': 'description', 'type': 'STRING'}, {'property': 'tags', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}], 'Container': [{'property': 'name', 'type': 'STRING'}, {'property': 'tags', 'type': 'STRING'}, {'property': 'description', 'type': 'STRING'}], 'Component': [{'property': 'name', 'type': 'STRING'}, {'property': 'tags', 'type': 'STRING'}, {'property': 'description', 'type': 'STRING'}]}, 'rel_props': {'Uses': [{'property': 'description', 'type': 'STRING'}]}, 'relationships': [{'start': 'Person', 'type': 'Uses', 'end': 'Container'}, {'start': '

### Ask a predefined question

Ask a predefined question and the predefined query should be used to generate the answer.

```json
    {
        "question": "what software systems are used by customers",
        "query": "MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Customer' RETURN DISTINCT s.name,s.description",
    }
```

In [54]:
response = chain.invoke("What software systems are used by customers?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Customer'
RETURN DISTINCT s.name, s.description
Full Context:
[{'s.name': 'ATM', 's.description': 'Allows customers to withdraw cash.'}, {'s.name': 'Internet Banking System', 's.description': 'Allows customers to view information about their bank accounts, and make payments.'}]

> Finished chain.


{'query': 'What software systems are used by customers?',
 'result': 'Customers use ATM software for withdrawing cash and Internet Banking System software for viewing information about their bank accounts and making payments.'}

### Ask a question that is not in the predefined list

The QA Chain should be able to find the similar question in the predefined list and use the corresponding query to generate the answer.

In [35]:
response = chain.invoke("What software systems are used by bank staff?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Bank Staff' RETURN DISTINCT s.name,s.description
Full Context:
[{'s.name': 'Mainframe Banking System', 's.description': 'Stores all of the core banking information about customers, accounts, transactions, etc.'}]

> Finished chain.


{'query': 'What software systems are used by bank staff?',
 'result': 'The Mainframe Banking System is used by bank staff to store all of the core banking information about customers, accounts, and transactions.'}

In [37]:
response = chain.invoke("what software systems can be used to make payments?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE s.description CONTAINS 'make payments' RETURN DISTINCT s.name, s.description
Full Context:
[{'s.name': 'Internet Banking System', 's.description': 'Allows customers to view information about their bank accounts, and make payments.'}]

> Finished chain.


{'query': 'what software systems can be used to make payments?',
 'result': 'An Internet Banking System can be used to make payments.'}

In [38]:
response = chain.invoke("what software systems can be used to withdraw cash?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:SoftwareSystem) WHERE s.description CONTAINS 'withdraw cash' RETURN DISTINCT s.name, s.description
Full Context:
[{'s.name': 'ATM', 's.description': 'Allows customers to withdraw cash.'}]

> Finished chain.


{'query': 'what software systems can be used to withdraw cash?',
 'result': 'An ATM (Automated Teller Machine) software system can be used to withdraw cash.'}

### Semantic Search

The cypher statements generated for the above examples are using simple matching against keywords, e.g. `p.tags contains 'Bank Staff'` 
Question :
`What software systems are used by bank staff?`

Generated Cypher:
`MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Bank Staff' RETURN DISTINCT s.name,s.description`

Result: 
`The Mainframe Banking System is used by bank staff to store all of the core banking information about customers, accounts, and transactions.`

if we ask the question in another way, QA Chain will not be able to answer correctly, it should use semnatic search instead to resolve the problem. 

https://python.langchain.com/docs/use_cases/graph/semantic/


In [56]:
response = chain.invoke("what software systems have banking information?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:SoftwareSystem) WHERE s.description CONTAINS 'banking information' RETURN DISTINCT s.name, s.description
Full Context:
[{'s.name': 'Mainframe Banking System', 's.description': 'Stores all of the core banking information about customers, accounts, transactions, etc.'}]

> Finished chain.


{'query': 'what software systems have banking information?',
 'result': 'The Mainframe Banking System stores all of the core banking information.'}

In [57]:
response = chain.invoke("what software systems can be used to view customer information?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:Uses]->(s:SoftwareSystem) WHERE p.tags contains 'Customer' AND s.description CONTAINS 'customer information' RETURN DISTINCT s.name,s.description
Full Context:
[]

> Finished chain.


{'query': 'what software systems can be used to view customer information?',
 'result': "I'm unable to provide an answer without specific software systems mentioned in the information provided."}

In [30]:
# https://python.langchain.com/docs/integrations/text_embedding/mistralai/
# https://python.langchain.com/docs/integrations/vectorstores/neo4jvector/

from langchain_community.vectorstores import Neo4jVector
# from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_mistralai import MistralAIEmbeddings


# https://api.python.langchain.com/en/latest/_modules/langchain_community/vectorstores/neo4j_vector.html#Neo4jVector.from_existing_graph
# Initialize and return a Neo4jVector instance from an existing graph.
vectorestore = Neo4jVector.from_existing_graph(
    embedding=MistralAIEmbeddings(mistral_api_key=os.getenv("MISTRAL_API_KEY")),
    node_label="SoftwareSystem",
    embedding_node_property="embedding",
    text_node_properties=["name", "description"],
    index_name="SoftwareSystem_index",
    url=os.getenv("NEO4J_URI"), username=os.getenv("NEO4J_USERNAME"), password=os.getenv("NEO4J_PASSWORD")
)

result = vectorestore.similarity_search("Slovenia", k=1)


ValidationError: 1 validation error for MistralAIEmbeddings
__root__
  An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on. (type=value_error.localentrynotfound; response=None; request=None)